In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import spacy
from spacy.matcher import DependencyMatcher
nlp = spacy.load('en_core_web_lg')
matcher = DependencyMatcher(nlp.vocab)

In [3]:
dramas = pd.read_csv('review_resolve_emoji.csv', index_col = 0)
dramas.head()

,title,review
0,Goblin,"If you are hesitating to watch this at all, pl..."
1,Goblin,I feel compelled to write a review for Goblin ...
2,Goblin,It must be said that Goblin is very much a som...
3,Goblin,Every moment of it indeed shined! Goblin is a ...
4,Goblin,"Somewhere inside Goblin, buried under a mound ..."


- create a main couple dictionary for look up

In [4]:
cp = pd.read_csv('main_couple.csv', index_col = 0)
cp.head()

,Title,Ratings,Ranked,Popularity,Name,Gender,Image,Character1
0,100 Days My Prince,8.3,#354,#160,Do Kyung Soo,Male,https://i.mydramalist.com/43e46c.jpg,Crown Prince Lee Yool / Na Won Deuk
1,100 Days My Prince,8.3,#354,#160,Nam Ji Hyun,Female,https://i.mydramalist.com/wlJ8Yc.jpg,Hong Shim / Yoon Yi Seo
2,18 Again,8.8,#44,#287,Kim Ha Neul,Female,https://i.mydramalist.com/Qy7kW_5c.jpg,Hong Dae Young [37 years old]
3,18 Again,8.8,#44,#287,Yoon Sang Hyun,Male,https://i.mydramalist.com/dxN6W_5c.jpg,Jung Da Jung
4,7th Grade Civil Servant,6.9,#4405,#632,Joo Won,Male,https://i.mydramalist.com/wrJg1_5c.jpg,Han Gil Ro / Han Pil Hoon


In [5]:
name = list(cp['Name'].values)
name = [''.join(nam.split()) for nam in name]
CPtitle = list(cp['Title'].values)
i = 0
cp_dict = {}
while i < len(cp):
    cp_dict[CPtitle[i]] = (name[i], name[i+1])
    i+=2

In [6]:
cp_dict

{'100 Days My Prince': ('DoKyungSoo', 'NamJiHyun'),
 '18 Again': ('KimHaNeul', 'YoonSangHyun'),
 '7th Grade Civil Servant': ('JooWon', 'ChoiKangHee'),
 'A Beautiful Mind': ('JangHyuk', 'ParkSoDam'),
 "A Gentleman's Dignity": ('JangDongGun', 'KimHaNeul'),
 'A Korean Odyssey': ('OhYeonSeo', 'LeeSeungGi'),
 'A Love So Beautiful': ('SoJuYeon', 'KimYoHan'),
 'A Piece of Your Mind': ('ChaeSooBin', 'JungHaeIn'),
 'A Poem a Day': ('LeeYooBi', 'LeeJoonHyuk'),
 'About Time': ('LeeSungKyung', 'LeeSangYoon'),
 'Abyss': ('ParkBoYoung', 'AhnHyoSeop'),
 'All About My Romance': ('LeeMinJung', 'ShinHaKyun'),
 'Angel Eyes': ('LeeSangYoon', 'GuHyeSeon'),
 "Angel's Last Mission: Love": ('KimMyungSoo', 'ShinHyeSun'),
 'Another Miss Oh': ('EricMun', 'SeoHyunJin'),
 'Arang and the Magistrate': ('ShinMinAh', 'LeeJoonGi'),
 'Are You Human Too?': ('SeoKangJoon', 'GongSeungYeon'),
 'Arthdal Chronicles Part 1: The Children of Prophecy': ('SongJoongKi',
  'KimJiWon'),
 'Arthdal Chronicles Part 3: The Prelude To Al

- read roles information

In [7]:
roles = pd.read_csv('roles_mod.csv', index_col = 0)
roles = roles.fillna('')
roles.head()

,Title,CharName,Role,ActorName,Gender,Image
0,When Time Stopped,KimSunAh,Main Role,AhnJiHyun,Female,https://i.mydramalist.com/0NwROc.jpg
1,My Only Love Song,MuMyung,Support Role,AhnBoHyun,Male,https://i.mydramalist.com/EoxeQ_5c.jpg
2,Itaewon Class,"JangGeunWon,Jangga'ssuccessor]",Support Role,AhnBoHyun,Male,https://i.mydramalist.com/EoxeQ_5c.jpg
3,Her Private Life,NamEunGi,Support Role,AhnBoHyun,Male,https://i.mydramalist.com/EoxeQ_5c.jpg
4,Wednesday 3:30 PM,BaekSeungGyu,Main Role,AhnBoHyun,Male,https://i.mydramalist.com/EoxeQ_5c.jpg


- use aggregate to put all individual reviews into one

In [8]:
dramas = dramas.groupby('title', sort=False).aggregate({'review':'sum'})
titles = list(dramas.index)

- create a dict for each pair

In [9]:
couple_dict = {v:[] for k, v in cp_dict.items()}
for title in titles:
    df = roles[roles['Title']==title]
    actorsList = list(df['ActorName'])
    for i in range(len(actorsList)):
        for j in range(i+1, len(actorsList)):
            if (actorsList[i], actorsList[j]) not in couple_dict and (actorsList[j], actorsList[i]) not in couple_dict:
                couple_dict[(actorsList[i], actorsList[j])] = []

- for each drama, create a cast list and look up dict (whenever we match the character name, we can know the corresponding actor name)

In [10]:
def casts_dict(df): # take in df = roles[roles['Title'==title]] as argument
    char = list(df['CharName'].values)
    casts = list(df['ActorName'])
    lookUp_dict = {}
    i=0
    for ch in char:
        temp = ch.split(',')
        lookUp_dict[casts[i]]=temp
        if ch != '':
            casts += temp
        i+=1
    return lookUp_dict, casts

- define patterns to match. Keywords = ['chemistry', 'relationship', 'OTP (one true pairing)', 'ship', 'root (for)', 'romance', 'bromance']

In [11]:
# chemistry
pattern1 = [
    {
        "RIGHT_ID": "chemistry",
        "RIGHT_ATTRS": {"LOWER": "chemistry"}
    },
    {
        "LEFT_ID": "chemistry",
        "REL_OP": ">", #'.*''
        "RIGHT_ID": "chem_prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "chem_prep",
        "REL_OP": ">",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    },
    {
        "LEFT_ID": "object",
        "REL_OP": ">",
        "RIGHT_ID": "conjunct",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "compound"]}}, #conj: between A and B; amod: between the male cast; compound: between Goblin couple
    }
]# chem between A and B
pattern2 = [
    {
        "RIGHT_ID": "chem_root",
        "RIGHT_ATTRS": {"LEMMA": {"IN": ["have", "share", "illustrate"]}}
    },
    {
        "LEFT_ID": "chem_root",
        "REL_OP": ">",
        "RIGHT_ID": "subject",
        "RIGHT_ATTRS": {"DEP": "nsubj"},
    },
    {
        "LEFT_ID": "chem_root",
        "REL_OP": ">", 
        "RIGHT_ID": "chemistry",
        "RIGHT_ATTRS": {"LOWER": "chemistry", "DEP": "dobj"},
    },
    {
        "LEFT_ID": "chemistry",
        "REL_OP": ">", 
        "RIGHT_ID": "chem_prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "chem_prep",
        "REL_OP": ">",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    }
]# A have chem with B
pattern3 = [
    {
        "RIGHT_ID": "chem_root",
        "RIGHT_ATTRS": {"LEMMA": {"IN": ["have", "share", "illustrate"]}}
    },
    {
        "LEFT_ID": "chem_root",
        "REL_OP": ">",
        "RIGHT_ID": "chemistry",
        "RIGHT_ATTRS": {"LOWER": "chemistry", "DEP": "dobj"},
    },
    {
        "LEFT_ID": "chem_root",
        "REL_OP": ">",
        "RIGHT_ID": "subject",
        "RIGHT_ATTRS": {"DEP": "nsubj"},
    },
    {
        "LEFT_ID": "subject",
        "REL_OP": ">",
        "RIGHT_ID": "conjunct",
        "RIGHT_ATTRS": {"DEP": {"IN": ["amod", "compound", "conj"]}},
    }
]# A and B have chem
pattern4 = [
    {
        "RIGHT_ID": "chemistry",
        "RIGHT_ATTRS": {"LOWER": "chemistry"}
    },
    {
        "LEFT_ID": "chemistry",
        "REL_OP": ">", 
        "RIGHT_ID": "obj1",
        "RIGHT_ATTRS": {"DEP": "nmod"},
    },
    {
        "LEFT_ID": "obj1",
        "REL_OP": ">", 
        "RIGHT_ID": "conj",
        "RIGHT_ATTRS": {"DEP": "conj"},
    }
] # A and B's chem
pattern5 = [
    {
        "RIGHT_ID": "chemistry",
        "RIGHT_ATTRS": {"LOWER": "chemistry"}
    },
    {
        "LEFT_ID": "chemistry",
        "REL_OP": ">", 
        "RIGHT_ID": "chem_poss",
        "RIGHT_ATTRS": {"DEP": "poss"},
    },
    {
        "LEFT_ID": "chem_poss",
        "REL_OP": ">", 
        "RIGHT_ID": "conj",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "compound"]}},
    }
]
# main lead's chemistry
pattern6 = [
    {
        "RIGHT_ID": "chemistry",
        "RIGHT_ATTRS": {"LOWER": "chemistry"}
    },
    {
        "LEFT_ID": "chemistry",
        "REL_OP": ">", #'.*''
        "RIGHT_ID": "chem_poss",
        "RIGHT_ATTRS": {"DEP": {"IN": ["poss", "compound", "amod"]}},
    },
    {
        "LEFT_ID": "chemistry",
        "REL_OP": ">", 
        "RIGHT_ID": "chem_prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "chem_prep",
        "REL_OP": ">", 
        "RIGHT_ID": "obj",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    }
] # A('s) chem with B
pattern7 = [
    {
        "RIGHT_ID": "chemistry",
        "RIGHT_ATTRS": {"LOWER": "chemistry"}
    },
    {
        "LEFT_ID": "chemistry",
        "REL_OP": ">", 
        "RIGHT_ID": "clause",
        "RIGHT_ATTRS": {"DEP": "relcl"},
    },
    {
        "LEFT_ID": "clause",
        "REL_OP": ">",
        "RIGHT_ID": "subject",
        "RIGHT_ATTRS": {"DEP": "nsubj"},
    },
    {
        "LEFT_ID": "subject",
        "REL_OP": ">",
        "RIGHT_ID": "conjunct",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "compound", "amod"]}}, #conj: A and B; amod: male cast; compound: Goblin couple
    }
]# the chemistry A and B have was amazing
pattern8 = [
    {
        "RIGHT_ID": "chemistry",
        "RIGHT_ATTRS": {"LOWER": "chemistry"}
    },
    {
        "LEFT_ID": "chemistry",
        "REL_OP": ">", 
        "RIGHT_ID": "chem_poss",
        "RIGHT_ATTRS": {"DEP": {"IN": ["compound", "poss"]}},
    },
    {
        "LEFT_ID": "chem_poss",
        "REL_OP": ";*", 
        "RIGHT_ID": "and",
        "RIGHT_ATTRS": {"LOWER": "and"},
    },
    {
        "LEFT_ID": "and",
        "REL_OP": "<",
        "RIGHT_ID": "conj",
        "RIGHT_ATTRS": {"ENT_TYPE": "PERSON"},
    }
] # A and B('s) chem
pattern9 = [
    {
        "RIGHT_ID": "chemistry",
        "RIGHT_ATTRS": {"ORTH": "chemistry"}
    },
    {
        "LEFT_ID": "chemistry",
        "REL_OP": ">", 
        "RIGHT_ID": "compound",
        "RIGHT_ATTRS": {"DEP": "compound"},
    },
    {
        "LEFT_ID": "chemistry",
        "REL_OP": ">", 
        "RIGHT_ID": "amod",
        "RIGHT_ATTRS": {"DEP": "amod"},
    }
] # main leads chem
matcher.add("CHEMISTRY1", [pattern1])
matcher.add("CHEMISTRY2", [pattern2])
matcher.add("CHEMISTRY3", [pattern3])
matcher.add("CHEMISTRY4", [pattern4])
matcher.add("CHEMISTRY5", [pattern5])
matcher.add("CHEMISTRY6", [pattern6])
matcher.add("CHEMISTRY7", [pattern7])
matcher.add("CHEMISTRY8", [pattern8])
matcher.add("CHEMISTRY9", [pattern9])

In [12]:
# relationship
pattern1 = [
    {
        "RIGHT_ID": "relationship",
        "RIGHT_ATTRS": {"LOWER": "relationship"}
    },
    {
        "LEFT_ID": "relationship",
        "REL_OP": ">", #'.*''
        "RIGHT_ID": "rel_prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "rel_prep",
        "REL_OP": ">",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    },
    {
        "LEFT_ID": "object",
        "REL_OP": ">",
        "RIGHT_ID": "conjunct",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "compound"]}}, #conj: between A and B; amod: between the male cast; compound: between Goblin couple
    }
]# relationship between/of A and B
pattern2 = [
    {
        "RIGHT_ID": "relationship",
        "RIGHT_ATTRS": {"LOWER": "relationship"}
    },
    {
        "LEFT_ID": "relationship",
        "REL_OP": ">", 
        "RIGHT_ID": "obj1",
        "RIGHT_ATTRS": {"DEP": "nmod"},
    },
    {
        "LEFT_ID": "obj1",
        "REL_OP": ">", 
        "RIGHT_ID": "conj",
        "RIGHT_ATTRS": {"DEP": "conj"},
    }
] # A and B's relationship
pattern3 = [
    {
        "RIGHT_ID": "relationship",
        "RIGHT_ATTRS": {"LOWER": "relationship"}
    },
    {
        "LEFT_ID": "relationship",
        "REL_OP": ">", 
        "RIGHT_ID": "rel_poss",
        "RIGHT_ATTRS": {"DEP": "poss"},
    },
    {
        "LEFT_ID": "rel_poss",
        "REL_OP": ">", 
        "RIGHT_ID": "conj",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "nummod", "compound"]}},
    }
] # main couple's relationship
pattern4 = [
    {
        "RIGHT_ID": "relationship",
        "RIGHT_ATTRS": {"LOWER": "relationship"}
    },
    {
        "LEFT_ID": "relationship",
        "REL_OP": ">", #'.*''
        "RIGHT_ID": "rel_poss",
        "RIGHT_ATTRS": {"DEP": {"IN": ["poss", "compound", "amod"]}},
    },
    {
        "LEFT_ID": "relationship",
        "REL_OP": ">", 
        "RIGHT_ID": "rel_prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "rel_prep",
        "REL_OP": ">", 
        "RIGHT_ID": "obj",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    }
] # A('s) relationship with B
pattern5 = [
    {
        "RIGHT_ID": "relationship",
        "RIGHT_ATTRS": {"LOWER": "relationship"}
    },
    {
        "LEFT_ID": "relationship",
        "REL_OP": ">", 
        "RIGHT_ID": "rel_poss",
        "RIGHT_ATTRS": {"DEP": {"IN": ["compound", "poss"]}},
    },
    {
        "LEFT_ID": "rel_poss",
        "REL_OP": ";*", 
        "RIGHT_ID": "and",
        "RIGHT_ATTRS": {"ORTH": "and"},
    },
    {
        "LEFT_ID": "and",
        "REL_OP": "<",
        "RIGHT_ID": "conj",
        "RIGHT_ATTRS": {"ENT_TYPE": "PERSON"},
    }
] # A and B('s) relationship (improper: Sunny and GrimReaper's)
pattern6 = [
    {
        "RIGHT_ID": "relationship",
        "RIGHT_ATTRS": {"LOWER": "relationship"}
    },
    {
        "LEFT_ID": "relationship",
        "REL_OP": ">", 
        "RIGHT_ID": "compound",
        "RIGHT_ATTRS": {"DEP": "compound"},
    },
    {
        "LEFT_ID": "relationship",
        "REL_OP": ">", 
        "RIGHT_ID": "amod",
        "RIGHT_ATTRS": {"DEP": "amod"},
    }
] # main leads relationship (improper)
matcher.add("RELATIONSHIP1", [pattern1])
matcher.add("RELATIONSHIP2", [pattern2])
matcher.add("RELATIONSHIP3", [pattern3])
matcher.add("RELATIONSHIP4", [pattern4])
matcher.add("RELATIONSHIP5", [pattern5])
matcher.add("RELATIONSHIP6", [pattern6])

In [13]:
# OTP
pattern1 = [
    {
        "RIGHT_ID": "OTP",
        "RIGHT_ATTRS": {"LOWER": "otp"}
    },
    {
        "LEFT_ID": "OTP",
        "REL_OP": "<", #'.*''
        "RIGHT_ID": "be",
        "RIGHT_ATTRS": {"LEMMA": "be"}
    },
    {
        "LEFT_ID": "be",
        "REL_OP": ">",
        "RIGHT_ID": "A",
        "RIGHT_ATTRS": {"DEP": "nsubj"},
    },
    {
        "LEFT_ID": "A",
        "REL_OP": ">",
        "RIGHT_ID": "B",
        "RIGHT_ATTRS": {"DEP": "conj"}, #conj: between A and B; amod: between the male cast; compound: between Goblin couple
    }
]# A and B are OTP
pattern2 = [
    {
        "RIGHT_ID": "A",
        "RIGHT_ATTRS": {"ENT_TYPE": "PERSON"}
    },
    {
        "LEFT_ID": "A",
        "REL_OP": ">", #'.*''
        "RIGHT_ID": "OTP",
        "RIGHT_ATTRS": {"DEP": {"IN": ["amod", "nmod", "compound"], "LOWER": "otp"}},
    },
    {
        "LEFT_ID": "A",
        "REL_OP": ">",
        "RIGHT_ID": "B",
        "RIGHT_ATTRS": {"DEP": "conj"},
    }
]# OTP A and B
pattern3 = [
    {
        "RIGHT_ID": "OTP",
        "RIGHT_ATTRS": {"LOWER": "otp"}
    },
    {
        "LEFT_ID": "OTP",
        "REL_OP": ">",
        "RIGHT_ID": "B",
        "RIGHT_ATTRS": {"DEP": "poss"},
    },
    {
        "LEFT_ID": "B",
        "REL_OP": ";*",
        "RIGHT_ID": "and",
        "RIGHT_ATTRS": {"ORTH": "and"},
    },
    {
        "LEFT_ID": "and",
        "REL_OP": "<",
        "RIGHT_ID": "A",
        "RIGHT_ATTRS": {"ENT_TYPE": "PERSON"},
    }
]# A and B's OTP(optional)
pattern4 = [
    {
        "RIGHT_ID": "otp",
        "RIGHT_ATTRS": {"LOWER": "otp"}
    },
    {
        "LEFT_ID": "otp",
        "REL_OP": ">",
        "RIGHT_ID": "prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "prep",
        "REL_OP": ">",
        "RIGHT_ID": "B",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    },
    {
        "LEFT_ID": "B",
        "REL_OP": ">",
        "RIGHT_ID": "A",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "compound"]}},
    }
]# otp between A and B
pattern5 = [
    {
        "RIGHT_ID": "A",
        "RIGHT_ATTRS": {"ENT_TYPE": "PERSON"}
    },
    {
        "LEFT_ID": "A",
        "REL_OP": ">",
        "RIGHT_ID": "B",
        "RIGHT_ATTRS": {"DEP": "conj"}, 
    },
    {
        "LEFT_ID": "A",
        "REL_OP": ">",
        "RIGHT_ID": "as",
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": "as"},
    },
    {
        "LEFT_ID": "as",
        "REL_OP": ">",
        "RIGHT_ID": "OTP",
        "RIGHT_ATTRS": {"DEP": "pobj", "LOWER": "otp"},
    }
]# A and B as OTP
matcher.add("OTP1", [pattern1])
matcher.add("OTP2", [pattern2])
matcher.add("OTP3", [pattern3])
matcher.add("OTP4", [pattern4])
matcher.add("OTP5", [pattern5])

In [14]:
# ship
pattern1 = [
    {
        "RIGHT_ID": "ship",
        "RIGHT_ATTRS": {"LOWER": "ship", "POS": "VERB"}
    },
    {
        "LEFT_ID": "ship",
        "REL_OP": ">", #'.*''
        "RIGHT_ID": "B",
        "RIGHT_ATTRS": {"DEP": "dobj"}
    },
    {
        "LEFT_ID": "B",
        "REL_OP": ">",
        "RIGHT_ID": "A",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "compound"]}},
    }
]# (I) ship A and B
pattern2 = [
    {
        "RIGHT_ID": "ship",
        "RIGHT_ATTRS": {"LOWER": "ship", "POS": "VERB"}
    },
    {
        "LEFT_ID": "ship",
        "REL_OP": ">", #'.*''
        "RIGHT_ID": "A",
        "RIGHT_ATTRS": {"DEP": "dobj"},
    },
    {
        "LEFT_ID": "ship",
        "REL_OP": ">", #'.*''
        "RIGHT_ID": "prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "prep",
        "REL_OP": ">",
        "RIGHT_ID": "B",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    }
]# ship A with B
pattern3 = [
    {
        "RIGHT_ID": "is",
        "RIGHT_ATTRS": {"LEMMA": "be"}
    },
    {
        "LEFT_ID": "is",
        "REL_OP": ">",
        "RIGHT_ID": "ship",
        "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": "ship"},
    },
    {
        "LEFT_ID": "is",
        "REL_OP": ">",
        "RIGHT_ID": "A",
        "RIGHT_ATTRS": {"DEP": "attr"},
    },
    {
        "LEFT_ID": "A",
        "REL_OP": ">",
        "RIGHT_ID": "B",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "compound"]}},
    }
]# (my) ship is A and B
pattern4 = [
    {
        "RIGHT_ID": "A",
        "RIGHT_ATTRS": {"ENT_TYPE": "PERSON"}
    },
    {
        "LEFT_ID": "A",
        "REL_OP": ".*",
        "RIGHT_ID": "B",
        "RIGHT_ATTRS": {"ENT_TYPE": "PERSON"},
    },
    {
        "LEFT_ID": "B",
        "REL_OP": ".",
        "RIGHT_ID": "ship",
        "RIGHT_ATTRS": {"LOWER": "ship", "POS": "NOUN"},
    }
]# the A and B ship
pattern5 = [
    {
        "RIGHT_ID": "ship",
        "RIGHT_ATTRS": {"LOWER": "ship", "POS": "NOUN"}
    },
    {
        "LEFT_ID": "ship",
        "REL_OP": ">",
        "RIGHT_ID": "prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "prep",
        "REL_OP": ">",
        "RIGHT_ID": "B",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    },
    {
        "LEFT_ID": "B",
        "REL_OP": ">",
        "RIGHT_ID": "A",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "compound"]}},
    }
]# the ship with A and B
matcher.add("SHIP1", [pattern1])
matcher.add("SHIP2", [pattern2])
matcher.add("SHIP3", [pattern3])
matcher.add("SHIP4", [pattern4])
matcher.add("SHIP5", [pattern5])

In [15]:
# root for
pattern1 = [
    {
        "RIGHT_ID": "root",
        "RIGHT_ATTRS": {"LOWER": "root", "POS": "VERB"}
    },
    {
        "LEFT_ID": "root",
        "REL_OP": ">", #'.*''
        "RIGHT_ID": "prep",
        "RIGHT_ATTRS": {"DEP": "prep"}
    },
    {
        "LEFT_ID": "prep",
        "REL_OP": ">", #'.*''
        "RIGHT_ID": "B",
        "RIGHT_ATTRS": {"DEP": "pobj"}
    },
    {
        "LEFT_ID": "B",
        "REL_OP": ">",
        "RIGHT_ID": "A",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "compound"]}},
    }
]# root for A and B/main couple
matcher.add("ROOT", [pattern1])

In [16]:
#bromance
pattern1 = [
    {
        "RIGHT_ID": "bromance",
        "RIGHT_ATTRS": {"LOWER": "bromance"}
    },
    {
        "LEFT_ID": "bromance",
        "REL_OP": ">",
        "RIGHT_ID": "prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "prep",
        "REL_OP": ">",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    },
    {
        "LEFT_ID": "object",
        "REL_OP": ">",
        "RIGHT_ID": "conjunct",
        "RIGHT_ATTRS": {"DEP": "conj"},
    }
] #bromance between/of/with A and B [3,2]

pattern2 = [
    {
        "RIGHT_ID": "had",
        "RIGHT_ATTRS": {"LEMMA": "have"},
    },
    {
        "LEFT_ID": "had",
        "REL_OP": ">",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": "nsubj"},
    },
    {
        "LEFT_ID": "object",
        "REL_OP": ">",
        "RIGHT_ID": "object1",
        "RIGHT_ATTRS": {"DEP": "conj"},
    },
    {
        "LEFT_ID": "had",
        "REL_OP": ">",
        "RIGHT_ID": "bromance",
        "RIGHT_ATTRS": {"DEP": "dobj", "LOWER": "bromance"},
    }
] # A and B had bromance [1,2]

pattern3 = [
    {
        "RIGHT_ID": "bromance",
        "RIGHT_ATTRS": {"LOWER": "bromance"},
    },
    {
        "LEFT_ID": "bromance",
        "REL_OP": ">",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": {"IN": ["compound", "poss"]}},
    },
    {
        "LEFT_ID": "bromance",
        "REL_OP": ">",
        "RIGHT_ID": "with",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "with",
        "REL_OP": ">",
        "RIGHT_ID": "object2",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    },
] # A('s) bromance with B [1,3]                   bromance[40]

pattern4 = [
    {
        "RIGHT_ID": "bromance",
        "RIGHT_ATTRS": {"LOWER": "bromance"},
    },
    {
        "LEFT_ID": "bromance",
        "REL_OP": ">",
        "RIGHT_ID": "bromance_poss",
        "RIGHT_ATTRS": {"DEP": "poss"},
    },
    {
        "LEFT_ID": "bromance",
        "REL_OP": "<",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": "conj"},
    },
] # A and B's bromance [1,2]                       bromance[50]

pattern5 = [
    {
        "RIGHT_ID": "bromance",
        "RIGHT_ATTRS": {"LOWER": "bromance"},
    },
    {
        "LEFT_ID": "bromance",
        "REL_OP": ">",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": "compound"},
    },
    {
        "LEFT_ID": "object",
        "REL_OP": ">",
        "RIGHT_ID": "conjunct",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "compound"]}},
    },
] # A-B bromance [2,1]                                bromance[88]
matcher.add("BROMANCE1", [pattern1])
matcher.add("BROMANCE2", [pattern2])
matcher.add("BROMANCE3", [pattern3])
matcher.add("BROMANCE4", [pattern4])
matcher.add("BROMANCE5", [pattern5])

In [17]:
#romance
pattern1 = [
    {
        "RIGHT_ID": "romance",
        "RIGHT_ATTRS": {"LOWER": "romance"},
    },
    {
        "LEFT_ID": "romance",
        "REL_OP": ">",
        "RIGHT_ID": "romance_prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "romance_prep",
        "REL_OP": ">",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    },
    {
        "LEFT_ID": "object",
        "REL_OP": ">",
        "RIGHT_ID": "conjunct",
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "compound"]}},
    }
] # romance between A and B [3,2]                                   romance[30]

pattern2 = [
    {
        "RIGHT_ID": "romance",
        "RIGHT_ATTRS": {"LOWER": "romance"},
    },
    {
        "LEFT_ID": "romance",
        "REL_OP": ">",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": "nmod"},
    },
    {
        "LEFT_ID": "object",
        "REL_OP": ">",
        "RIGHT_ID": "conjunct", 
        "RIGHT_ATTRS": {"DEP": {"IN": ["conj", "amod", "compound"]}},
    },
] # A and B romance [2,1]                                              romance[78]

pattern3 = [
    {
        "RIGHT_ID": "romance",
        "RIGHT_ATTRS": {"LOWER": "romance"},
    },
    {
        "LEFT_ID": "romance",
        "REL_OP": ">",
        "RIGHT_ID": "romance_prep",
        "RIGHT_ATTRS": {"DEP": "prep"},
    },
    {
        "LEFT_ID": "romance_prep",
        "REL_OP": ">",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": "pobj"},
    },
    {
        "LEFT_ID": "romance",
        "REL_OP": ">",
        "RIGHT_ID": "obj1",
        "RIGHT_ATTRS": {"DEP": "poss"},
    }
] # A's romance with B [2,3]
matcher.add("ROMANCE1", [pattern1])
matcher.add("ROMANCE2", [pattern2])
matcher.add("ROMANCE3", [pattern3])

- define find_obj functions: for every match, take the objects and add that to couple_dict for sentiment analysis

In [18]:
def find_obj(txt, title):
    doc = nlp(txt)
    matches = matcher(doc)
    for match in matches:
        match_id, token_ids = match
        if nlp.vocab.strings[match_id]=='CHEMISTRY1':
            char1 = str(doc[token_ids[3]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='CHEMISTRY2':
            char1 = str(doc[token_ids[1]])
            char2 = str(doc[token_ids[4]])
        elif nlp.vocab.strings[match_id]=='CHEMISTRY3':
            char1 = str(doc[token_ids[3]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='CHEMISTRY4':
            char1 = str(doc[token_ids[2]])
            char2 = str(doc[token_ids[1]])
        elif nlp.vocab.strings[match_id]=='CHEMISTRY5':
            char1 = str(doc[token_ids[2]])
            char2 = str(doc[token_ids[1]])
        elif nlp.vocab.strings[match_id]=='CHEMISTRY6':
            char1 = str(doc[token_ids[1]])
            char2 = str(doc[token_ids[3]])
        elif nlp.vocab.strings[match_id]=='CHEMISTRY7':
            char1 = str(doc[token_ids[3]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='CHEMISTRY8':
            char1 = str(doc[token_ids[1]])
            char2 = str(doc[token_ids[3]])
        elif nlp.vocab.strings[match_id]=='CHEMISTRY9':
            char1 = str(doc[token_ids[2]])
            char2 = str(doc[token_ids[1]])
        elif nlp.vocab.strings[match_id]=='RELATIONSHIP1':
            char1 = str(doc[token_ids[3]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='RELATIONSHIP2':
            char1 = str(doc[token_ids[2]])
            char2 = str(doc[token_ids[1]])
        elif nlp.vocab.strings[match_id]=='RELATIONSHIP3':
            char1 = str(doc[token_ids[2]])
            char2 = str(doc[token_ids[1]])
        elif nlp.vocab.strings[match_id]=='RELATIONSHIP4':
            char1 = str(doc[token_ids[1]])
            char2 = str(doc[token_ids[3]])
        elif nlp.vocab.strings[match_id]=='RELATIONSHIP5':
            char1 = str(doc[token_ids[1]])
            char2 = str(doc[token_ids[3]])
        elif nlp.vocab.strings[match_id]=='RELATIONSHIP6':
            char1 = str(doc[token_ids[2]])
            char2 = str(doc[token_ids[1]])
        elif nlp.vocab.strings[match_id]=='OTP1':
            char1 = str(doc[token_ids[2]])
            char2 = str(doc[token_ids[3]])
        elif nlp.vocab.strings[match_id]=='OTP2':
            char1 = str(doc[token_ids[0]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='OTP3':
            char1 = str(doc[token_ids[1]])
            char2 = str(doc[token_ids[3]])
        elif nlp.vocab.strings[match_id]=='OTP4':
            char1 = str(doc[token_ids[3]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='OTP5':
            char1 = str(doc[token_ids[0]])
            char2 = str(doc[token_ids[1]])
        elif nlp.vocab.strings[match_id]=='SHIP1':
            char1 = str(doc[token_ids[2]])
            char2 = str(doc[token_ids[1]])
        elif nlp.vocab.strings[match_id]=='SHIP2':
            char1 = str(doc[token_ids[1]])
            char2 = str(doc[token_ids[3]])
        elif nlp.vocab.strings[match_id]=='SHIP3':
            char1 = str(doc[token_ids[3]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='SHIP4':
            char1 = str(doc[token_ids[0]])
            char2 = str(doc[token_ids[1]])
        elif nlp.vocab.strings[match_id]=='SHIP5':
            char1 = str(doc[token_ids[3]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='BROMANCE1':
            char1 = str(doc[token_ids[3]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='BROMANCE2':
            char1 = str(doc[token_ids[1]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='BROMANCE3':
            char1 = str(doc[token_ids[1]])
            char2 = str(doc[token_ids[3]])
        elif nlp.vocab.strings[match_id]=='BROMANCE4':
            char1 = str(doc[token_ids[1]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='BROMANCE5':
            char1 = str(doc[token_ids[2]])
            char2 = str(doc[token_ids[1]])
        elif nlp.vocab.strings[match_id]=='ROMANCE1':
            char1 = str(doc[token_ids[3]])
            char2 = str(doc[token_ids[2]])
        elif nlp.vocab.strings[match_id]=='ROMANCE2':
            char1 = str(doc[token_ids[2]])
            char2 = str(doc[token_ids[1]])
        elif nlp.vocab.strings[match_id]=='ROMANCE3':
            char1 = str(doc[token_ids[2]])
            char2 = str(doc[token_ids[3]])
        else: # match root for
            char1 = str(doc[token_ids[3]])
            char2 = str(doc[token_ids[2]])
            
        if char1 not in lookUp_dict: #char1 is not a dict key
            for actor, charList in lookUp_dict.items():
                if char1 in charList:
                    char1 = actor # if it is in the charList, after looking up the dict, we change it to corresponding actor's name, else, it stays the same
        if char2 not in lookUp_dict: #char2 is not a dict key
            for actor, charList in lookUp_dict.items():
                if char2 in charList:
                    char2 = actor
            
        check = char1+" "+char2
        if check.lower() in ['main casts', 'main leads', 'main couple']:
            if title in cp_dict: # for the dramas which we know the main couples, we can add
                char1, char2 = cp_dict[title]
  
        if (char1, char2) in couple_dict:
            if txt not in couple_dict[(char1, char2)]:
                couple_dict[(char1, char2)] = couple_dict[(char1, char2)] + [txt]
        elif (char2, char1) in couple_dict:
            if txt not in couple_dict[(char2, char1)]:
                couple_dict[(char2, char1)] = couple_dict[(char2, char1)] + [txt]

In [19]:
for title in titles:
    ruler = nlp.add_pipe("entity_ruler", before='ner')
    text = dramas.loc[title, 'review']
    lines = re.split(';\s*|:\s*|!\s*|\?\s*|\.\s*', text)
    lines = [line for line in lines if not line.isspace() if line]
    
    chemistry = [line for line in lines if "CHEMISTRY" in line or "Chemistry" in line or 'chemistry' in line]
    OTP = [line for line in lines if "OTP" in line or "otp" in line]
    relation = [line for line in lines if "RELATIONSHIP" in line or "Relationship" in line or 'relationship' in line]
    ship = [line for line in lines if " SHIP " in line or " Ship " in line or ' ship ' in line]
    root = [line for line in lines if "ROOT" in line or "Root" in line or 'root' in line]
    romance = [line for line in lines if " ROMANCE " in line or " Romance " in line or ' romance ' in line]
    bromance = [line for line in lines if " BROMANCE " in line or " Bromance " in line or ' bromance ' in line]
    
    df = roles[roles['Title']==title]
    lookUp_dict, casts = casts_dict(df)
    for cast in casts:
        ruler.add_patterns([{"label": "PERSON", "pattern": cast}])
    for chem in chemistry:
        find_obj(chem, title)
    for otp in OTP:
        find_obj(otp, title)
    for rel in relation:
        find_obj(rel, title)
    for s in ship:
        find_obj(s, title)
    for r in root:
        find_obj(r, title)
    for rom in romance:
        find_obj(rom, title)
    for brom in bromance:
        find_obj(brom, title)
    
    nlp.remove_pipe('entity_ruler')

In [20]:
count = 0
for tup, List in couple_dict.items():
    if len(List) > 0:
        print(tup, List)
        print('\n')
        count+=1

('DoKyungSoo', 'NamJiHyun') ['Also the chemistry between NamJiHyun and DoKyungSoo was amazing', 'Actress NamJiHyun does most of the heavy lifting that makes the relationship between HongShim and CrownPrinceLeeYool work even a little']


('KimHaNeul', 'YoonSangHyun') ["Also, it's worth saying the relationship between the main couple is not the only one well-developed during this drama"]


('JooWon', 'ChoiKangHee') ['I enjoyed the chemistry that JooWon and ChoiKangHee had b/c the drama had such a relaxed feel to /c']


('JangHyuk', 'ParkSoDam') ["ParkSoDam also has pretty good chemistry with JangHyuk , and some really cute scenes, but there's no strong spark between them", "ParkSoDam and JangHyuk had really good chemistry, even though I also enjoyed JangHyuk 's and ParkSeYoung interactions in the beginning (yeah, part of it was Money Flower-induced, ", 'Also, for the development of the relationship between LeeYoungOh and GyeJinSung , especially how she is slowly changing LeeYoungOh']


(

In [21]:
count

415

In [22]:
from spacytextblob.spacytextblob import SpacyTextBlob

nlp.add_pipe('spacytextblob')
pair = []
actor1 = []
actor2 = []
lineCount = []
score = []
for k, v in couple_dict.items():
    pair.append(k)
    actor1.append(k[0])
    actor2.append(k[1])
    lineCount.append(len(v))
    text = '. '.join(v)
    doc = nlp(text)
    score.append(doc._.polarity)

In [23]:
pair_df = pd.DataFrame({'pair': pair, 'actor1': actor1, 'actor2': actor2, 
                        'lineCount': lineCount, 'score': score})

In [24]:
drop = list(pair_df[pair_df['lineCount']<5].index)
final = pair_df.drop(drop).sort_values('score', ascending=False).reset_index(drop=True)

In [25]:
final

,pair,actor1,actor2,lineCount,score
0,"(ChaSeungWon, LeeSeungGi)",ChaSeungWon,LeeSeungGi,5,0.592857
1,"(JungHaeIn, SonYeJin)",JungHaeIn,SonYeJin,9,0.501087
2,"(YoonHyunMin, GoSungHee)",YoonHyunMin,GoSungHee,5,0.487549
3,"(SongJiHyo, ChoiJinHyuk)",SongJiHyo,ChoiJinHyuk,5,0.478333
4,"(JungEunJi, KimJiSoo)",JungEunJi,KimJiSoo,6,0.466667
...,...,...,...,...,...
98,"(JoJungSuk, ParkBoYoung)",JoJungSuk,ParkBoYoung,8,0.105357
99,"(YooAhIn, KimHeeAe)",YooAhIn,KimHeeAe,5,0.098148
100,"(ShinHyeSun, LeeJongSuk)",ShinHyeSun,LeeJongSuk,5,0.076746
101,"(KimGoEun, ParkHaeJin)",KimGoEun,ParkHaeJin,8,0.025000


In [26]:
def find_actor(name):
    return pd.concat([final[final['actor1']==name], final[final['actor2']==name]], axis = 0)

In [31]:
find_actor('ParkBoYoung')

,pair,actor1,actor2,lineCount,score
27,"(ParkBoYoung, ParkHyungShik)",ParkBoYoung,ParkHyungShik,15,0.319820
53,"(ParkBoYoung, AhnHyoSeop)",ParkBoYoung,AhnHyoSeop,6,0.258410
98,"(JoJungSuk, ParkBoYoung)",JoJungSuk,ParkBoYoung,8,0.105357


In [28]:
final.to_csv('senti.csv')

In [32]:
couple_dict[('ChaSeungWon', 'LeeSeungGi')] #from 'A Korean Odyssey'

['The chemistry between WooMaWang and SonOhGong , the "bromance", is really enjoyable and brings like 50% of the comedy in the show',
 "The end to Boo Ja s story I won't say here, but if you've seen it you'll know why I'm upset The inconsistency of the bromance/friendships I loved the relationship between SonOhGong and WooMaWang but it felt really inconsistent to me",
 'I also liked the bromance between SonOhGong and WooMaWang ',
 "I loved LeeSeungGi , the bromance between SonOhGong and WooMaWang , the powers and the fight with heaven, One's character, and Pal Gye and Bu Ja's flirting",
 "However, the bromance between WooMaWang and SonOhGong is probably the best one I've seen since becoming a K-drama fan"]